# Information Retrieval

This notebook demonstrates how to put together a simple retrieval engine in python. We'll focus on *ranked retrieval*, using a TFxIDF vector space model. We won't bother about any optimisations, and just use python dicts, sets etc.

Our dataset is the *Cranfield collection*, an early resource which has been widely used in IR research. You may need to call `nltk.download()` to install the stopwords corpus. 

In [18]:
import nltk
from math import log
from collections import defaultdict, Counter
import requests, tarfile

First, download the cranfield files, using the code below. Or you can do this manually by downloading http://ir.dcs.gla.ac.uk/resources/test_collections/cran/cran.tar.gz and extracting the files into the current directory.

In [19]:
# download
fname = 'cran.tar.gz'
url = 'http://ir.dcs.gla.ac.uk/resources/test_collections/cran/' + fname
r = requests.get(url)
# write to disk
open(fname , 'wb').write(r.content)
# extract files from archive
tar = tarfile.open(fname, "r:")
tar.extractall()
tar.close()

We'll need to extract the document details---the document ids and text content---from the cranfield files. 

In [20]:
def parse_cranfield_documents(fname):
    SKIP, READ = range(2)
    state = SKIP
    identifier = body = None
    with open(fname) as infile:
        for line in infile:
            line = line.strip()
            parts = line.split()
            first = parts[0]
            if first[0] == ".":
                if first[1] == "I":
                    if state != None and identifier:
                        yield (identifier, body)
                    identifier = int(parts[1])
                    state = SKIP
                elif first[1] == "W":
                    state = READ
                    body = ''
                else:
                    state = SKIP
            elif state == READ:
                body += line + ' '
        if state == READ:
            yield (identifier, body)

In [21]:
docfname = 'cran.all.1400'
raw_docs = list(parse_cranfield_documents(docfname))
len(raw_docs)
raw_docs[0]

(1,
 'experimental investigation of the aerodynamics of a wing in a slipstream . an experimental study of a wing in a propeller slipstream was made in order to determine the spanwise distribution of the lift increase due to slipstream at different angles of attack of the wing and at different free stream to slipstream velocity ratios .  the results were intended in part as an evaluation basis for different theoretical treatments of this problem . the comparative span loading curves, together with supporting evidence, showed that a substantial part of the lift increment produced by the slipstream was due to a /destalling/ or boundary-layer-control effect .  the integrated remaining lift increment, after subtracting this destalling lift, was found to agree well with a potential flow theory . an empirical evaluation of the destalling effects was made for the specific configuration of the experiment . ')

In [22]:
print(raw_docs[-1])

(1400, 'the buckling shear stress of simply-supported infinitely long plates with transverse stiffeners . this report is an extension of previous theoretical investigations of the elastic buckling in shear of flat plates reinforced by transverse stiffeners . the plates are treated as infinitely long and simply-supported along the long sides . stiffeners are spaced at regular intervals, dividing the plate into a number of panels of uniform size . the effect ob bending and torsional stiffnesses of the stiffener upon the buckling shear stress is calculated for the complete range of stiffnesses, for panels with ratios of width to stiffener spacing of graphical forms . ')


We'll need to tokenise the documents, remove stop-words and stem the words to form our bag-of-words representation. Here we'll use the PorterStemmer (there are others in NLTK). 

In [23]:
stopwords = set(nltk.corpus.stopwords.words('english')) # wrap in a set() (see below)
stemmer = nltk.stem.PorterStemmer() 

def extract_terms(doc):
    terms = set()
    for token in nltk.word_tokenize(doc):
        if token not in stopwords: # 'in' and 'not in' operations are much faster over sets that lists
            terms.add(stemmer.stem(token.lower()))
    return terms

Let's test the method using the first document.

In [24]:
print(list(sorted(extract_terms(raw_docs[0][1]))))

[',', '.', '/destalling/', 'aerodynam', 'agre', 'angl', 'attack', 'basi', 'boundary-layer-control', 'compar', 'configur', 'curv', 'destal', 'determin', 'differ', 'distribut', 'due', 'effect', 'empir', 'evalu', 'evid', 'experi', 'experiment', 'flow', 'found', 'free', 'increas', 'increment', 'integr', 'intend', 'investig', 'lift', 'load', 'made', 'order', 'part', 'potenti', 'problem', 'produc', 'propel', 'ratio', 'remain', 'result', 'show', 'slipstream', 'span', 'spanwis', 'specif', 'stream', 'studi', 'substanti', 'subtract', 'support', 'theoret', 'theori', 'togeth', 'treatment', 'veloc', 'well', 'wing']


We probably want to remove numbers and punctuation, which aren't being caught by the stop list. We may want to be a bit more agressive with tokenising hyphenated words (although take care, as some might be important.) Have a go yourself and see if you can improve the preprocessing to correct for these issues.

Now we can apply the term extraction method to all documents in our corpus. (This may take a minute.) 

In [25]:
docs = {}
for docid, text in raw_docs:
    terms = extract_terms(text)
    docs[docid] = terms

To build a _vector space model_ we must define a scoring function capable of relating a query to a document, then use this to retrieve the top *k* scoring documents for a given query. The scoring function we use is the cosine similarity measure over a _TF*IDF_ vector representation of the document and the query. 

This requires us to process the data to compute:
1. term frequencies within each document (a *bag* of words, rather than a *set* as for boolean retrieval)
2. document frequencies for each term, counting how many documents they occur in
3. normlised _tf*idf_ vectors for each document 
4. an inverted index to allow for efficient lookup by term

The first step is to collect term frequencies for each document, which is only a slight change from the code above for creating the binary index.

In [26]:
def extract_term_freqs(doc):
    tfs = Counter()
    for token in doc:
        if token not in stopwords: # 'in' and 'not in' operations are much faster over sets that lists
            tfs[stemmer.stem(token.lower())] += 1
    return tfs

We'll also need to compute the document frequencies, for the *idf* factors in the scoring function.

In [27]:
def compute_doc_freqs(doc_term_freqs):
    dfs = Counter()
    for tfs in doc_term_freqs.values():
        for term in tfs.keys():
            dfs[term] += 1
    return dfs

Using the above two functions, process the corpus into term frequencies and document frequencies.

In [28]:
doc_term_freqs = {}
for docid, terms in docs.items():
    term_freqs = extract_term_freqs(terms)
    doc_term_freqs[docid] = term_freqs
M = len(doc_term_freqs)

In [29]:
doc_freqs = compute_doc_freqs(doc_term_freqs)

As a sanity check, let's check some of the *df* values:

In [30]:
for term in "flux viscous magnet".split():
    stem = stemmer.stem(term.lower())
    print("term", term, "df", doc_freqs[stem], 'idf', log(M / float(doc_freqs[stem])))

term flux df 16 idf 4.471638793363569
term viscous df 117 idf 2.482053580805594
term magnet df 37 idf 3.6333096029591254


Now to create the inverted index. For this we need to include the scoring function to compute the *tf* and *idf* values, then normalise each document vector to be unit length. For this we process each document in the corpus, compute a vector with one entry per term, normalise for length and store in the inverted index.

Note that this a fairly inefficient inverted index. Many optimisations are possible, and are indeed necessary for this to scale to more realistic sized datasets. The approach below, though, is fine for small collections like this one. 

In [31]:
vsm_inverted_index = defaultdict(list)
for docid, term_freqs in doc_term_freqs.items():
    N = sum(term_freqs.values())
    length = 0
    
    # find tf*idf values and accumulate sum of squares 
    tfidf_values = []
    for term, count in term_freqs.items():
        tfidf = float(count) * log(M / float(doc_freqs[term]))
        tfidf_values.append((term, tfidf))
        length += tfidf ** 2

    # normalise documents by length and insert into index
    length = length ** 0.5
    for term, tfidf in tfidf_values:
        # note the inversion of the indexing, to be term -> (doc_id, score)
        vsm_inverted_index[term].append([docid, tfidf / length])
        
# ensure posting lists are in sorted order (less important here cf above)
for term, docids in vsm_inverted_index.items():
    docids.sort()

For querying this index, we use the algorithm from the lecture. This just sums up the weights for each document using the posting lists for all query terms, then returns the ranked list of results. (Again, there are more efficient algorithms for doing ranked retrieval in the VSM, such as [WAND](http://lucene.472066.n3.nabble.com/attachment/577044/0/p426-broder.pdf).)

In [32]:
def query_vsm(query, index, k=10):
    accumulator = Counter()
    for term in query:
        postings = index[term]
        for docid, weight in postings:
            accumulator[docid] += weight
    return accumulator.most_common(k)

results = query_vsm([stemmer.stem(term.lower()) for term in "flux viscous magnet".split()], vsm_inverted_index)
results

[(298, 0.0009694331486352642),
 (208, 0.0009694244092723588),
 (1253, 0.0009693369541745064),
 (653, 0.0009693353264029864),
 (967, 0.000969333754779725),
 (112, 0.0009693326008957937),
 (1273, 0.0009693318118730736),
 (299, 0.0009693314295824477),
 (267, 0.0009693277699471529),
 (1250, 0.0008955698481602709)]

It's worth taking a look at the top scoring document(s), e.g.

In [33]:
print(raw_docs[results[0][0]-1])

(298, 'incompressible wedge flows of an electrically conducting viscous fluid in the presence of a magnetic field . the purpose of this note is to discuss the two-dimensional flow of an electrically conducting viscous fluid past a wedge in the presence of a magnetic field .  the governing differential equations and boundary conditions are given and analyzed . ')


Note that this document does not include all the query terms. While it has *viscous* and *magnet* it does not include *flux*.

This collection contains other files, besides the lists of documents. Namely a list of sample queries, and judgements of the relevance of each document against these queries. We'll return to this in later weeks when covering IR evaluation.